In [25]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

In [26]:
# Directory config to project root to insure consistency across environments for project specific imports
from pyprojroot import here
os.chdir(here())

# Project specific imports
from src.utils import save_dataframe


## Data Quering and Loading

In [27]:
# DB related imports
# from database.db_utils import init_db
# from config.config_loader import load_config
# from database.queries import prepped_data_query
# from sqlalchemy import text

# # Initialize local PostgreSQL session
# Session = init_db(load_config("DB_URL"))
# session = Session()

# # Execute and fetch results
# data = session.execute(text(prepped_data_query))

# # Close the session
# session.close()

# # Convert to DataFrame
# data = pd.DataFrame(data.fetchall(), columns=data.keys())

In [28]:
# Saving cleaned dataset for future use
# save_dataframe(data, "00_base_data.csv")

In [81]:
# Load dataset
data = pd.read_csv("./data/00_base_data.csv")

## Initial Data Overview

In [30]:
# data["missing_budget"] = (data["budget"] == 0).astype(int)
# data["missing_revenue"] = (data["revenue"] == 0).astype(int)

In [31]:
# data["budget"] = data["budget"].replace(0, np.nan)
# data["revenue"] = data["revenue"].replace(0, np.nan)

In [32]:
# data = data[(data["budget"] != 0) & (data["revenue"] != 0)]

In [33]:
# data = data[(data["budget"] != 0) | (data["revenue"] != 0)]

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3003 entries, 0 to 3002
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    3003 non-null   object 
 1   release_date             3003 non-null   object 
 2   tmdb_vote_count          3003 non-null   int64  
 3   tmdb_vote_average        3003 non-null   float64
 4   genre_names              3003 non-null   object 
 5   budget                   1594 non-null   float64
 6   revenue                  1816 non-null   float64
 7   runtime_in_min           3001 non-null   float64
 8   tmdb_popularity          3003 non-null   float64
 9   production_company_name  2990 non-null   object 
 10  production_country_name  2998 non-null   object 
 11  spoken_languages         3001 non-null   object 
 12  director                 3001 non-null   object 
 13  writer                   2972 non-null   object 
 14  actors                  

In [35]:
# Count missing values
data.isnull().sum()

title                         0
release_date                  0
tmdb_vote_count               0
tmdb_vote_average             0
genre_names                   0
budget                     1409
revenue                    1187
runtime_in_min                2
tmdb_popularity               0
production_company_name      13
production_country_name       5
spoken_languages              2
director                      2
writer                       31
actors                        6
imdb_rating                   8
imdb_votes                    6
metascore                   781
age_rating                  276
awards                      564
rotten_tomatoes_rating      467
meta_critic_rating          781
dtype: int64

In [36]:
data.head(5)

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,director,writer,actors,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating
0,Carter,2022-08-05,378,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",5.1,"10,297",NaN,TV-MA,NaN,32.0,NaN
1,The Shadow in My Eye,2021-10-28,494,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",7.3,"17,277",NaN,TV-MA,2 wins & 13 nominations,NaN,NaN
2,Noelle,2019-11-12,673,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",6.3,"28,616",48.0,G,1 win total,56.0,48.0
3,Play,2019-09-04,349,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",7.2,"1,755",NaN,NaN,1 nomination,NaN,NaN
4,Dave Chappelle: Sticks & Stones,2019-08-26,320,7.600,Comedy,NaN,NaN,65.0,7.683,NaN,...,Stan Lathan,Dave Chappelle,Dave Chappelle,8.3,"28,457",NaN,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.0,NaN


## Multilable Categorical Features

There are a nuymber of multilable categorical features that will need to be looked into.

In [37]:
def count_unique_values_for_feature(df: pd.DataFrame, feature: str, delimiter: str = ",") -> int:
    """
    Splits the specified feature column by the delimiter and returns the number of unique values.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        feature (str): The name of the column to process.
        delimiter (str): The delimiter used to separate multiple values in the column.

    Returns:
        int: The number of unique values.
    """
    return len(df[feature].dropna().str.split(rf"{delimiter}\s*").explode().unique())

# List of features you want to analyze:
features = [
    "genre_names", 
    "production_company_name", "production_country_name", 
            "spoken_languages", "director", "writer", "actors"]

# Create a dictionary with the counts for each feature:
unique_counts = {feature: count_unique_values_for_feature(data, feature) for feature in features}

# Display the results:
for feature, count in unique_counts.items():
    print(f"{feature}: {count} unique values")

genre_names: 19 unique values
production_company_name: 4358 unique values
production_country_name: 83 unique values
spoken_languages: 89 unique values
director: 2251 unique values
writer: 4593 unique values
actors: 4961 unique values


In [38]:
def print_top_categories(df: pd.DataFrame, column: str, top_n: int, delimiter: str = ",", others_label: str = "Others") -> None:
    """
    Prints the top_n unique values from a multi-label column and the total count of values 
    that fall outside the top_n (which would be grouped as 'Others').

    Args:
        df (pd.DataFrame): The DataFrame containing your data.
        column (str): The name of the multi-label column.
        top_n (int): The number of top categories to display.
        delimiter (str): The delimiter separating multiple values (default is a comma).
        others_label (str): The label used for less frequent values.
    """
    # Split the column into individual values and count frequencies
    exploded = df[column].dropna().str.split(rf"{delimiter}\s*").explode().str.strip()
    counts = exploded.value_counts()
    
    # Get the top N categories and the sum for the rest
    top_categories = counts.head(top_n)
    others_count = counts[counts.index.difference(top_categories.index)].sum()
    
    print("--------------------------------------------------||")
    print(f"Top {top_n} unique values for '{column}':")
    print(top_categories)
    print(f"Total count of all other values (will be grouped as '{others_label}'): {others_count}")
    print("--------------------------------------------------||\n")


top_values = {
    "genre_names": 20,
    "production_company_name": 20,
    "production_country_name": 10,
    "spoken_languages": 10,
    "director": 20,
    "writer": 20,
    "actors": 20
}

for feature, top_n in top_values.items():
    print_top_categories(data, feature, top_n)

--------------------------------------------------||
Top 20 unique values for 'genre_names':
genre_names
Drama              1292
Comedy              976
Thriller            864
Action              735
Horror              514
Adventure           487
Romance             401
Crime               393
Science Fiction     388
Fantasy             371
Family              301
Mystery             300
Animation           294
History             189
War                  92
Music                89
Documentary          59
TV Movie             29
Western              27
Name: count, dtype: int64
Total count of all other values (will be grouped as 'Others'): 0
--------------------------------------------------||

--------------------------------------------------||
Top 20 unique values for 'production_company_name':
production_company_name
Universal Pictures          109
Lionsgate                    91
Columbia Pictures            81
Paramount Pictures           77
Warner Bros. Pictures        76
Walt 

In [39]:
def group_top_categories(df: pd.DataFrame, column: str, top_n: int, delimiter: str = ",", others_label: str = "Others") -> pd.Series:
    """
    Processes a multi-label column by keeping only the top_n categories (based on frequency) 
    and replacing any other category with the 'others_label'.

    Args:
        df (pd.DataFrame): The DataFrame containing your data.
        column (str): The name of the multi-label column.
        top_n (int): The number of top categories to keep.
        delimiter (str): The delimiter separating multiple values in the column.
        others_label (str): The label to use for all categories not in the top_n.

    Returns:
        pd.Series: A new Series with the modified values.
    """
    # Split the column and count frequencies
    exploded = df[column].dropna().str.split(rf"{delimiter}\s*").explode().str.strip()
    counts = exploded.value_counts()
    top_categories = counts.head(top_n).index.tolist()
    
    def map_categories(cell):
        if pd.isna(cell):
            return cell
        cats = [cat.strip() for cat in cell.split(delimiter)]
        # Replace any category not in top_categories with others_label
        new_cats = [cat if cat in top_categories else others_label for cat in cats]
        # Remove duplicates while preserving order
        seen = set()
        new_cats = [x for x in new_cats if x not in seen and not seen.add(x)]
        return delimiter.join(new_cats)
    
    return df[column].apply(map_categories)

top_values = {
    # "genre_names": 20,  # Keeping all genres
    # "production_company_name": 20,  # Too granular
    "production_country_name": 5,
    "spoken_languages": 5,
    # "director": 20,  # Too granular
    # "writer": 20,  # Too granular
    # "actors": 20  # Too granular
}

for feature, top_n in top_values.items():
    data[feature] = group_top_categories(data, feature, top_n)


In [40]:
data.head()

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,director,writer,actors,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating
0,Carter,2022-08-05,378,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",5.1,"10,297",NaN,TV-MA,NaN,32.0,NaN
1,The Shadow in My Eye,2021-10-28,494,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",7.3,"17,277",NaN,TV-MA,2 wins & 13 nominations,NaN,NaN
2,Noelle,2019-11-12,673,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",6.3,"28,616",48.0,G,1 win total,56.0,48.0
3,Play,2019-09-04,349,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",7.2,"1,755",NaN,NaN,1 nomination,NaN,NaN
4,Dave Chappelle: Sticks & Stones,2019-08-26,320,7.600,Comedy,NaN,NaN,65.0,7.683,NaN,...,Stan Lathan,Dave Chappelle,Dave Chappelle,8.3,"28,457",NaN,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.0,NaN


## Handling Nulls

### Simple Imputer

#### Categorical Imputer

In [41]:
from sklearn.impute import SimpleImputer
# Create a transformer for categorical features
cat_imputer = SimpleImputer(strategy="constant", fill_value="Unknown")

# List of your categorical columns where you want to replace nulls with "Unknown"
cat_cols = ['production_company_name', 'production_country_name', 'spoken_languages', 'director', 'writer', 'actors', 'age_rating']

In [42]:
# Apply imputer
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])
data.head(30)

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,director,writer,actors,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating
0,Carter,2022-08-05,378,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",5.1,"10,297",NaN,TV-MA,NaN,32.0,NaN
1,The Shadow in My Eye,2021-10-28,494,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",7.3,"17,277",NaN,TV-MA,2 wins & 13 nominations,NaN,NaN
2,Noelle,2019-11-12,673,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",6.3,"28,616",48.0,G,1 win total,56.0,48.0
3,Play,2019-09-04,349,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",7.2,"1,755",NaN,Unknown,1 nomination,NaN,NaN
4,Dave Chappelle: Sticks & Stones,2019-08-26,320,7.600,Comedy,NaN,NaN,65.0,7.683,Unknown,...,Stan Lathan,Dave Chappelle,Dave Chappelle,8.3,"28,457",NaN,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.0,NaN
5,"Neuilly sa mère, sa mère !",2018-08-08,332,5.337,Comedy,NaN,NaN,105.0,6.277,SND,...,"Gabriel Julien-Laferrière, Djamel Bensalah","Djamel Bensalah, Isaac Sharry, Marc de Chauveron","Denis Podalydès, Samy Seghir, Jérémy Denisty",5.2,518,NaN,Unknown,NaN,NaN,NaN
6,Ricky Gervais: Humanity,2018-03-13,461,7.700,Comedy,NaN,NaN,79.0,5.702,Unknown,...,John L. Spencer,Ricky Gervais,Ricky Gervais,8.0,"24,326",NaN,TV-MA,NaN,47.0,NaN
7,Kedi,2017-01-08,323,7.568,Documentary,NaN,5037923.0,79.0,7.949,Termite Films,...,Ceyda Torun,Unknown,"Yaman Barlas, Sari, Arzu Göl",7.6,"16,133",78.0,Not Rated,6 wins & 21 nominations,98.0,78.0
8,Wedding Unplanned,2017-04-26,469,6.100,"Romance, Comedy",NaN,NaN,94.0,8.383,"Mandarin & Compagnie, Gaumont",...,Reem Kherici,"Reem Kherici, Stéphane Kazandjian, Philippe La...","Reem Kherici, Nicolas Duvauchelle, Julia Piaton",6.1,"4,133",NaN,TV-14,NaN,NaN,NaN
9,Time Is Up,2021-09-09,677,6.500,"Romance, Drama",NaN,NaN,108.0,12.646,"Lotus Production, RAI, Voltage Pictures",...,Elisa Amoruso,"Marco Belardi, Patrizia Fiorellini, Elisa Amoruso","Bella Thorne, Benjamin Mascolo, Sebastiano Pig...",4.2,"3,955",NaN,Unknown,NaN,NaN,NaN


In [43]:
zero_imputer = SimpleImputer(strategy='constant', fill_value=0)
zero_cols = ['imdb_rating', 'imdb_votes']

In [49]:
# Apply imputer
data[zero_cols] = zero_imputer.fit_transform(data[zero_cols])
data.head(15)

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,director,writer,actors,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating
0,Carter,2022-08-05,378,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",5.1,"10,297",NaN,TV-MA,NaN,32.0,NaN
1,The Shadow in My Eye,2021-10-28,494,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",7.3,"17,277",NaN,TV-MA,2 wins & 13 nominations,NaN,NaN
2,Noelle,2019-11-12,673,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",6.3,"28,616",48.0,G,1 win total,56.0,48.0
3,Play,2019-09-04,349,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",7.2,"1,755",NaN,Unknown,1 nomination,NaN,NaN
4,Dave Chappelle: Sticks & Stones,2019-08-26,320,7.600,Comedy,NaN,NaN,65.0,7.683,Unknown,...,Stan Lathan,Dave Chappelle,Dave Chappelle,8.3,"28,457",NaN,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.0,NaN
5,"Neuilly sa mère, sa mère !",2018-08-08,332,5.337,Comedy,NaN,NaN,105.0,6.277,SND,...,"Gabriel Julien-Laferrière, Djamel Bensalah","Djamel Bensalah, Isaac Sharry, Marc de Chauveron","Denis Podalydès, Samy Seghir, Jérémy Denisty",5.2,518,NaN,Unknown,NaN,NaN,NaN
6,Ricky Gervais: Humanity,2018-03-13,461,7.700,Comedy,NaN,NaN,79.0,5.702,Unknown,...,John L. Spencer,Ricky Gervais,Ricky Gervais,8.0,"24,326",NaN,TV-MA,NaN,47.0,NaN
7,Kedi,2017-01-08,323,7.568,Documentary,NaN,5037923.0,79.0,7.949,Termite Films,...,Ceyda Torun,Unknown,"Yaman Barlas, Sari, Arzu Göl",7.6,"16,133",78.0,Not Rated,6 wins & 21 nominations,98.0,78.0
8,Wedding Unplanned,2017-04-26,469,6.100,"Romance, Comedy",NaN,NaN,94.0,8.383,"Mandarin & Compagnie, Gaumont",...,Reem Kherici,"Reem Kherici, Stéphane Kazandjian, Philippe La...","Reem Kherici, Nicolas Duvauchelle, Julia Piaton",6.1,"4,133",NaN,TV-14,NaN,NaN,NaN
9,Time Is Up,2021-09-09,677,6.500,"Romance, Drama",NaN,NaN,108.0,12.646,"Lotus Production, RAI, Voltage Pictures",...,Elisa Amoruso,"Marco Belardi, Patrizia Fiorellini, Elisa Amoruso","Bella Thorne, Benjamin Mascolo, Sebastiano Pig...",4.2,"3,955",NaN,Unknown,NaN,NaN,NaN


In [47]:
# median_imputer = SimpleImputer(strategy='median')
# median_cols = []

# # Apply imputer
# data[median_cols] = median_imputer.fit_transform(data[median_cols])
# data.head(30)

In [70]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor


# Initialize the IterativeImputer (using BayesianRidge as the default estimator)
iter_imputer = IterativeImputer(
    # estimator=BayesianRidge(),
    estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    max_iter=10,
    initial_strategy='median',
    n_nearest_features=None,
    imputation_order='ascending',
    random_state=42
    )

iter_cols = ['metascore', 'rotten_tomatoes_rating', 'meta_critic_rating']

In [ ]:
from sklearn.compose import make_column_selector

num_cols = make_column_selector(dtype_include=['number'])
num_cols = num_cols(data)
print(num_cols)


['tmdb_vote_count', 'tmdb_vote_average', 'runtime_in_min', 'tmdb_popularity', 'metascore', 'rotten_tomatoes_rating', 'meta_critic_rating', 'metascore_missing', 'rotten_tomatoes_rating_missing', 'meta_critic_rating_missing']


In [ ]:
predictor_cols = [
    'tmdb_vote_count', 'tmdb_vote_average', 'runtime_in_min', 'tmdb_popularity', 'metascore', 'rotten_tomatoes_rating', 
    'meta_critic_rating', 'metascore_missing', 'rotten_tomatoes_rating_missing', 'meta_critic_rating_missing',
    ]
# predictor_cols = ['tmdb_vote_count', 'tmdb_vote_average']
full_iter_cols = iter_cols + predictor_cols

In [51]:
# Create missing indicator columns for each score column
for col in iter_cols:
    data[col + "_missing"] = data[col].isnull().astype(int)

# data.head(20)

In [61]:
# Apply imputer
data[num_cols] = iter_imputer.fit_transform(data[num_cols])
data.head(30)

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating,metascore_missing,rotten_tomatoes_rating_missing,meta_critic_rating_missing
0,Carter,2022-08-05,378.0,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,5.1,"10,297",58.0,TV-MA,NaN,32.00000,58.0,1.0,0.0,1.0
1,The Shadow in My Eye,2021-10-28,494.0,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,7.3,"17,277",58.0,TV-MA,2 wins & 13 nominations,61.13853,58.0,1.0,1.0,1.0
2,Noelle,2019-11-12,673.0,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,6.3,"28,616",48.0,G,1 win total,56.00000,48.0,0.0,0.0,0.0
3,Play,2019-09-04,349.0,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,7.2,"1,755",58.0,Unknown,1 nomination,61.13853,58.0,1.0,1.0,1.0
4,Dave Chappelle: Sticks & Stones,2019-08-26,320.0,7.600,Comedy,NaN,NaN,65.0,7.683,Unknown,...,8.3,"28,457",58.0,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.00000,58.0,1.0,0.0,1.0
5,"Neuilly sa mère, sa mère !",2018-08-08,332.0,5.337,Comedy,NaN,NaN,105.0,6.277,SND,...,5.2,518,58.0,Unknown,NaN,61.13853,58.0,1.0,1.0,1.0
6,Ricky Gervais: Humanity,2018-03-13,461.0,7.700,Comedy,NaN,NaN,79.0,5.702,Unknown,...,8.0,"24,326",58.0,TV-MA,NaN,47.00000,58.0,1.0,0.0,1.0
7,Kedi,2017-01-08,323.0,7.568,Documentary,NaN,5037923.0,79.0,7.949,Termite Films,...,7.6,"16,133",78.0,Not Rated,6 wins & 21 nominations,98.00000,78.0,0.0,0.0,0.0
8,Wedding Unplanned,2017-04-26,469.0,6.100,"Romance, Comedy",NaN,NaN,94.0,8.383,"Mandarin & Compagnie, Gaumont",...,6.1,"4,133",58.0,TV-14,NaN,61.13853,58.0,1.0,1.0,1.0
9,Time Is Up,2021-09-09,677.0,6.500,"Romance, Drama",NaN,NaN,108.0,12.646,"Lotus Production, RAI, Voltage Pictures",...,4.2,"3,955",58.0,Unknown,NaN,61.13853,58.0,1.0,1.0,1.0


In [12]:
from sklearn.compose import ColumnTransformer
iter_impute = ColumnTransformer(
    transformers=[
        ('iter_impute', iter_imputer, iter_cols)
    ],
    remainder="passthrough"  # Unlisted columns are kept unchanged (if any)
)

In [13]:
# from sklearn.compose import ColumnTransformer

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_pipeline, numeric_features),
#         ('cat', categorical_pipeline, categorical_features),
#         ('drop_cols', 'drop', ['id', 'unnecessary_column'])
#     ],
#     remainder="passthrough"  # Unlisted columns are kept unchanged (if any)
# )

In [14]:
columns_to_drop = []

In [15]:
# First, build a ColumnTransformer that imputes the groups that do not use IterativeImputer.
process_nulls = ColumnTransformer(
    transformers=[
        ('cat', cat_imputer, cat_cols),
        ('zero', zero_imputer, zero_cols),
        # ('median', median_imputer, median_cols),
        # ('drop_cols', 'drop', columns_to_drop)
    ],
    remainder='passthrough'  # Keep other features (including those for iterative imputer) as is.
)

In [16]:
from sklearn.pipeline import Pipeline

# Then, create a pipeline that applies the preprocessor and then the iterative imputer to the remaining columns.
pipeline = Pipeline(steps=[
    ('preprocess', process_nulls),
    ('iter_impute', iter_impute)
    # ...followed by scaling and model training as needed.
])

In [ ]:
# Now, you can fit the pipeline on your training data:
pipeline.fit(data)
# And transform the data:
data = pipeline.transform(data)
data.head()

ValueError: Passing extra keyword arguments to ColumnTransformer.transform is only supported if enable_metadata_routing=True, which you can set using `sklearn.set_config`. See the User Guide <https://scikit-learn.org/stable/metadata_routing.html> for more details. Extra parameters passed are: {'copy'}

In [71]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import FunctionTransformer
# from sklearn.preprocessing import StandardScaler  # Uncomment if needed later
# from sklearn.preprocessing import OneHotEncoder   # Uncomment if needed later

# Define your column groups
# zero_cols = ['some_numeric_col1', 'some_numeric_col2']  # Impute missing values with 0
# median_cols = ['budget', 'revenue', 'runtime_in_min']    # Impute missing values with the median
# cat_cols = ['production_company_name', 'production_country_name', 'spoken_languages', 
#             'director', 'writer', 'actors']              # Impute missing categorical values with "Unknown"
            
# Columns you want to drop after imputation (for example, identifiers or irrelevant features)
# drop_cols = ['unnecessary_column1', 'unnecessary_column2']

# Create individual pipelines for each group:

zero_pipeline = Pipeline(steps=[
    ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0))
    # ('scaler', StandardScaler())  # Uncomment to scale these columns
])

median_pipeline = Pipeline(steps=[
    ('median_imputer', SimpleImputer(strategy='median'))
    # ('scaler', StandardScaler())  # Uncomment to scale these columns
])

cat_pipeline = Pipeline(steps=[
    ('cat_imputer', SimpleImputer(strategy='constant', fill_value='Unknown'))
    # ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Uncomment if you want one-hot encoding here
])

# impute_pipeline = Pipeline(steps=[
#     ('iter_impute', iter_imputer),
# ])

# Combine them in a ColumnTransformer.
# Any columns not mentioned in the transformers will be left unchanged due to remainder='passthrough'.
preprocessor = ColumnTransformer(
    transformers=[
        ('zero', zero_pipeline, zero_cols),
        ('median', median_pipeline, median_cols),
        ('cat', cat_pipeline, cat_cols)
    ],
    remainder='passthrough'
)

# Set output to pandas dataframe
preprocessor.set_output(transform='pandas')

# Define a function transformer to drop unwanted columns.
def drop_unwanted_columns(X):
    # Assuming X is a pandas DataFrame; errors='ignore' ensures that missing columns are skipped.
    return X.drop(columns=columns_to_drop, errors='ignore')

dropper = FunctionTransformer(drop_unwanted_columns)


# iter_cols = ['remainder__metascore', 'remainder__rotten_tomatoes_rating', 'remainder__meta_critic_rating']

# IterativeImputer applied only to selected columns
iter_imputer = ColumnTransformer(
    transformers=[
        ('iter', iter_imputer, iter_cols)
    ],
    remainder='passthrough'
)

# Set output to pandas dataframe
iter_imputer.set_output(transform='pandas')

# Now, build the full pipeline.
# Note: The IterativeImputer should be applied after the initial imputations, so it comes after the ColumnTransformer and dropper.
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('dropper', dropper),
    # ('iter_imputer', iter_imputer),
    # ('scaler', StandardScaler()),  # Optionally, scale after imputation.
    # ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Optionally, if you need additional encoding.
])

# Example usage:
# Assume 'data' is your DataFrame with the full set of features.
# You can then fit_transform the pipeline on your training data:
transformed_data = full_pipeline.fit_transform(data)

# Column adjustment
# new_columns = [col.split("__", 1)[-1] for col in transformed_data.columns]
# transformed_data.columns = new_columns
transformed_data.head()

# If you need the result back as a DataFrame, you can do:
# (Here, we assume that the preprocessor returns a DataFrame; if it returns an array, you'll have to set column names accordingly.)
# transformed_df = pd.DataFrame(transformed_data)
# transformed_df.head()


,zero__imdb_rating,zero__imdb_votes,cat__production_company_name,cat__production_country_name,cat__spoken_languages,cat__director,cat__writer,cat__actors,cat__age_rating,remainder__title,...,remainder__revenue,remainder__runtime_in_min,remainder__tmdb_popularity,remainder__metascore,remainder__awards,remainder__rotten_tomatoes_rating,remainder__meta_critic_rating,remainder__metascore_missing,remainder__rotten_tomatoes_rating_missing,remainder__meta_critic_rating_missing
0,5.1,"10,297",Apeitda,Others,"English,Others",Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",TV-MA,Carter,...,NaN,132.0,27.043,58.0,NaN,32.00000,58.0,1.0,0.0,1.0
1,7.3,"17,277",Miso Film,Others,Others,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",TV-MA,The Shadow in My Eye,...,NaN,107.0,16.008,58.0,2 wins & 13 nominations,61.13853,58.0,1.0,1.0,1.0
2,6.3,"28,616",Walt Disney Pictures,United States of America,"English,French",Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",G,Noelle,...,NaN,100.0,10.040,48.0,1 win total,56.00000,48.0,0.0,0.0,0.0
3,7.2,"1,755","Chapter 2, Moonshaker",France,French,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",Unknown,Play,...,NaN,108.0,5.214,58.0,1 nomination,61.13853,58.0,1.0,1.0,1.0
4,8.3,"28,457",Unknown,United States of America,English,Stan Lathan,Dave Chappelle,Dave Chappelle,TV-MA,Dave Chappelle: Sticks & Stones,...,NaN,65.0,7.683,58.0,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.00000,58.0,1.0,0.0,1.0


In [73]:
# Column adjustment
new_columns = [col.split("__", 1)[-1] for col in transformed_data.columns]
transformed_data.columns = new_columns

iter_pipeline = Pipeline(steps=[
    # ('preprocessor', preprocessor),
    # ('dropper', dropper),
    ('iter_imputer', iter_imputer),
    # ('scaler', StandardScaler()),  # Optionally, scale after imputation.
    # ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Optionally, if you need additional encoding.
])

transformed_data2 = iter_pipeline.fit_transform(transformed_data)

# Column adjustment
new_columns = [col.split("__", 1)[-1] for col in transformed_data2.columns]
transformed_data2.columns = new_columns

transformed_data2.head()

,metascore,rotten_tomatoes_rating,meta_critic_rating,imdb_rating,imdb_votes,production_company_name,production_country_name,spoken_languages,director,writer,...,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,awards,metascore_missing,rotten_tomatoes_rating_missing,meta_critic_rating_missing
0,58.0,32.00000,58.0,5.1,"10,297",Apeitda,Others,"English,Others",Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung",...,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,NaN,1.0,0.0,1.0
1,58.0,61.13853,58.0,7.3,"17,277",Miso Film,Others,Others,Ole Bornedal,Ole Bornedal,...,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,2 wins & 13 nominations,1.0,1.0,1.0
2,48.0,56.00000,48.0,6.3,"28,616",Walt Disney Pictures,United States of America,"English,French",Marc Lawrence,Marc Lawrence,...,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,1 win total,0.0,0.0,0.0
3,58.0,61.13853,58.0,7.2,"1,755","Chapter 2, Moonshaker",France,French,Anthony Marciano,"Max Boublil, Anthony Marciano",...,7.278,Comedy,NaN,NaN,108.0,5.214,1 nomination,1.0,1.0,1.0
4,58.0,42.00000,58.0,8.3,"28,457",Unknown,United States of America,English,Stan Lathan,Dave Chappelle,...,7.600,Comedy,NaN,NaN,65.0,7.683,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,1.0,0.0,1.0


### Custom Top N Transformer

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin

class TopCategoriesTransformer(BaseEstimator, TransformerMixin):
    """
    A custom transformer that groups multi-label categorical features by keeping the top N categories,
    replacing any other category with a specified label (e.g., "Others").
    """
    def __init__(self, column: str, top_n: int, delimiter: str = ",", others_label: str = "Others"):
        self.column = column
        self.top_n = top_n
        self.delimiter = delimiter
        self.others_label = others_label
        self.top_categories_ = None

    def fit(self, X, y=None):
        # Split and explode the column to get individual categories.
        exploded = X[self.column].dropna().str.split(rf"{self.delimiter}\s*").explode().str.strip()
        counts = exploded.value_counts()
        self.top_categories_ = counts.head(self.top_n).index.tolist()
        return self

    def transform(self, X):
        def map_categories(cell):
            if pd.isna(cell):
                return cell
            cats = [cat.strip() for cat in cell.split(self.delimiter)]
            # Replace categories not in top_categories_ with others_label.
            new_cats = [cat if cat in self.top_categories_ else self.others_label for cat in cats]
            # Remove duplicates while preserving order.
            seen = set()
            new_cats = [x for x in new_cats if x not in seen and not seen.add(x)]
            return self.delimiter.join(new_cats)
        
        X_transformed = X.copy()
        X_transformed[self.column] = X_transformed[self.column].apply(map_categories)
        return X_transformed

# Usage
genre_transformer = Pipeline(steps=[
    ('top_cat', TopCategoriesTransformer(column='production_country_name', top_n=5, delimiter=","))
])

In [79]:
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('top_n', iterative_pipeline, iterative_cols),
#         ('median', median_pipeline, median_cols),
#         ('zero', zero_pipeline, zero_cols)
#     ],
#     remainder='passthrough'  # Include the rest of the columns as they are
# )

In [82]:
top_categories_cols = ["production_country_name", "spoken_languages"]

# Apply imputer
data["production_country_name"] = genre_transformer.fit_transform(data["production_country_name"])
data.head(30)

KeyError: 'production_country_name'

In [83]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class GroupTopCategoriesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, top_values, delimiter=",", others_label="Others"):
        """
        Transformer for processing multi-label categorical columns by keeping only 
        the top_n categories and replacing others with a generic label.
        
        Args:
            top_values (dict): Dictionary mapping column names to their respective top_n value.
            delimiter (str): Delimiter used in multi-label columns.
            others_label (str): Label for less frequent categories.
        """
        self.top_values = top_values
        self.delimiter = delimiter
        self.others_label = others_label
        self.top_categories_dict = {}

    def fit(self, X, y=None):
        """
        Computes the top categories for each specified column based on frequency.
        """
        self.top_categories_dict = {}
        for column, top_n in self.top_values.items():
            if column in X:
                exploded = X[column].dropna().str.split(rf"{self.delimiter}\s*").explode().str.strip()
                counts = exploded.value_counts()
                self.top_categories_dict[column] = counts.head(top_n).index.tolist()
        return self

    def transform(self, X):
        """
        Applies the category grouping transformation.
        """
        X_transformed = X.copy()

        for column, top_categories in self.top_categories_dict.items():
            if column in X_transformed:
                X_transformed[column] = X_transformed[column].apply(self._map_categories, args=(top_categories,))
        
        return X_transformed

    def _map_categories(self, cell, top_categories):
        if pd.isna(cell):
            return cell
        cats = [cat.strip() for cat in cell.split(self.delimiter)]
        new_cats = [cat if cat in top_categories else self.others_label for cat in cats]
        seen = set()
        new_cats = [x for x in new_cats if x not in seen and not seen.add(x)]
        return self.delimiter.join(new_cats)


In [87]:
from sklearn.pipeline import Pipeline

top_values = {
    "production_country_name": 5,
    "spoken_languages": 5,
}

pipeline = Pipeline([
    ("group_top_categories", GroupTopCategoriesTransformer(top_values=top_values))
])

data_transformed = pipeline.fit_transform(data)
# data_transformed.head(30)

## Save cleanded data

In [88]:
# Saving cleaned dataset for future use
save_dataframe(data, "01_clean_data.csv")

✅ Data successfully saved to ./data\01_clean_data.csv with separator ','
